# Entrega 1 - Predictor de muerte de pacientes con HIV

### Grupo 26:
     - Agustín Martínez  C.I 5.074.743-0
     - Joaquín Mezquita  C.I 5.239.865-9
     - Nicolás Núñez     C.I 5.127.722-0
     



## 1. Objetivo

El objetivo de esta tarea es construir un algortimo capaz de predecir la muerte de pacientes bajo observación por HIV utilizando árboles de decisión generados con el algoritmo ID3 visto en el curso. Para ello, utilizaremos el dataset `AIDS Clinical Trials Group Study 175` tomando como valor a predecir el indicador de censura 'cid'. 

El indicador de censura puede tomar dos valores "censoring" o "failure". La censura ocurre cuando un valor de una observación solo se conoce parcialmente. 

En este caso ocurre cuando el experimento termina en un momento determinado, tras el cuál los pacientes todavía vivos quedan todos censurados por la derecha.  

Luego del momento de la última observación no se conocen datos sobre la muerte o supervivencia de los pacientes. Por lo que se puede tomar únicamente el caso "failure" como la muerte de un paciente.d


El éxito del aprendizaje se mide a través de...


## 2. Diseño del predictor

# Preprocesamiento

## Discretización del dataset

Antes de entrenar el arbol, se discretizan las variables que tomen valores númericos continuos, o valores discretos dentro de un intervalo amplio.

Tomando en cuenta las observaciones hechas a las columnas y los valores del dataset, estos atributos son: 

- pidnum
- time
- age
- wtkg
- karnof
- preanti
- cd40
- cd420
- cd80
- cd820

## Entrenamiento del predictor

Para evaluar la precisión del dataset, este es separado en 3 conjuntos (hay que usar evaluacion y ver bien que porcentajes usar)

- Entrenamiento / 80%
- Validación / 10%
- Evaluación / 10%



## Por ejemplo, para un clasificador:
    

## 2.1 Preprocesamiento de datos

Dado que el conjunto de datos no contiene campos con
- Decisiones sobre tratamiento de datos numéricos, faltantes, etc. antes de la aplicación de el algoritmo
- Selección/generación de atributos


## 2.2 Algoritmo
Extensiones del algoritmo original necesarias para la resolución del problema: tratamiento de atributos faltantes, numéricos, etc. (si es el propio algoritmo el que lo maneja), implementaciones adicionales necesarias para manejar ensambles de clasificadores, etc.


## 2.3 Evaluación
- Qué conjunto de métricas se utilizan para la evaluación de la solución y su definición
- Sobre qué conjunto(s) se realiza el entrenamiento, ajuste de la solución, evaluación, etc. Explicar cómo se construyen estos conjuntos.

## 3. Experimentación

- Presentar los distintos experimentos que se realizan y los resultados que se obtienen.

- La información de los resultados se presenta en tablas y en gráficos, de acuerdo a su naturaleza. Por ejemplo:

_En la gráfica 1, se observa el error cuadrático total del conjunto de entrenamiento a medida que pasan los juegos para el oponente X_

In [ ]:
import matplotlib.pyplot
matplotlib.pyplot.plot(range(0,100), [2**-(x/10) for x in range(0,100)])


- Debe existir alguna instancia donde se compile la información relevante de los experimentos de forma de poder comparar fácilmente los distintos experimentos. Por ejemplo:

_En la tabla 1, se presentan los distintos resultados contra el jugador aleatorio, para los distintos valores de $\alpha$ elegidos. El mejor resultado se obtiene para $\alpha=0.05$, lo que prueba que la estrategia..._

<table>
  <tr>
    <th>$\alpha$</th>
    <th>...</th>
    <th>Turnos</th>
    <th>Error</th>
  </tr>
  <tr>
    <td>0.001</td>
    <td>...</td>
    <td>100</td>
    <td>0.991</td>
  </tr>    
  <tr>
    <td>0.005</td>
    <td>...</td>
    <td>100</td>
    <td>0.987</td>
  </tr>
  <tr style="font-weight:bold">
    <td>0.05</td>
    <td>...</td>
    <td>100</td>
    <td>0.329</td>
  </tr>
  <tr>
    <td>0.5</td>
    <td>...</td>
    <td>100</td>
    <td>0.564</td>
  </tr>    
    <caption>Tabla 1 - Entrenamiento del jugador X para distintos valores de $\alpha$</caption>
</table>


## 4. Comparación

En esta sección se comparara nuetro algoritmo de ID3 contra los algoritmos de la libreria scikit-learn:
- DecisionTreeClassifier
- RandomForestClassifier

En principio, nuestro algoritmo muestra peores resultados que los ya implementados

## 5. Conclusión

Una breve conclusión del trabajo realizado. Por ejemplo: 
- ¿cuándo se dieron los mejores resultados del jugador?
- ¿encuentra alguna relación con los parámetros / oponentes/ atributos elegidos?
- ¿cómo mejoraría los resultados?